In [1]:
import sys
import os

from dotenv import load_dotenv
sys.path.insert(0, os.path.abspath('../src'))
sys.path.insert(0, os.path.abspath('../src/agents'))
print(sys.path)

load_dotenv()

['/Users/yannikhahn/Code/LLMsAgents-TextToSQL/src/agents', '/Users/yannikhahn/Code/LLMsAgents-TextToSQL/src', '/Users/yannikhahn/Code/LLMsAgents-TextToSQL/evaluation/src', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/yannikhahn/Library/Caches/pypoetry/virtualenvs/llmsagents-texttosql-JgGAmzxu-py3.12/lib/python3.12/site-packages']


True

In [2]:
import pandas as pd

# sample = pd.read_csv("../sample/dev.csv")
sample = pd.read_csv("../sample/spider.csv")

sample.head()

,db_id,SQL,query_toks,query_toks_no_value,question,question_toks,sql,question_id,execution_time
0,planet_1,"SELECT POSITION FROM Employee WHERE Name = ""...","['SELECT', 'POSITION', 'FROM', 'Employee', 'WH...","['select', 'position', 'from', 'employee', 'wh...",What is the position of Amy Wong?,"['What', 'is', 'the', 'position', 'of', 'Amy',...","{'from': {'table_units': [['table_unit', 0]], ...",1859,0.000005
1,institution_sports,SELECT T1.Nickname FROM championship AS T1 JOI...,"['SELECT', 'T1.Nickname', 'FROM', 'championshi...","['select', 't1', '.', 'nickname', 'from', 'cha...",What is the nickname of the institution with t...,"['What', 'is', 'the', 'nickname', 'of', 'the',...","{'from': {'table_units': [['table_unit', 1], [...",1659,0.000008
2,cre_Students_Information_Systems,"SELECT datetime_detention_start , datetime_de...","['SELECT', 'datetime_detention_start', ',', 'd...","['select', 'datetime_detention_start', ',', 'd...",Show the detention start time and end time of ...,"['Show', 'the', 'detention', 'start', 'time', ...","{'from': {'table_units': [['table_unit', 13]],...",477,0.000013
3,customers_and_orders,SELECT product_name FROM Products WHERE produc...,"['SELECT', 'product_name', 'FROM', 'Products',...","['select', 'product_name', 'from', 'products',...",Show all hardware type products in ascending o...,"['Show', 'all', 'hardware', 'type', 'products'...","{'from': {'table_units': [['table_unit', 0]], ...",259,0.000008
4,warehouse_1,SELECT sum(T1.value) FROM boxes AS T1 JOIN war...,"['SELECT', 'sum', '(', 'T1.value', ')', 'FROM'...","['select', 'sum', '(', 't1', '.', 'value', ')'...",What is the total value of boxes located in Ch...,"['What', 'is', 'the', 'total', 'value', 'of', ...","{'from': {'table_units': [['table_unit', 1], [...",1708,0.000007


In [3]:
sample.count()

db_id                  100
SQL                    100
query_toks             100
query_toks_no_value    100
question               100
question_toks          100
sql                    100
question_id            100
execution_time         100
dtype: int64

In [4]:
from langchain_ollama.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from agents.feedback_agent import FeedbackAgent
from prompt_templates.feedback_agent import ONE_SHOT, TWO_SHOT

model_name = "mistral"
dataset_name = "spider"
# llm = ChatOllama(model="mistral")
#llm = ChatOpenAI(model="gpt-4o-mini")
agent = FeedbackAgent(template=TWO_SHOT)
model_type = "two_shot"

In [ ]:
from tqdm.notebook import tqdm
import os

os.makedirs(f"./runs/feedback_agent/{dataset_name}/{model_name}", exist_ok=True)

try:
    ongoing_run = pd.read_csv(
        f"./runs/feedback_agent/{dataset_name}/{model_name}/{model_type}.csv"
    ).to_dict(orient="records")
    evaluation = ongoing_run
    start = len(ongoing_run)
except FileNotFoundError:
    evaluation = []
    start = 0

for row in tqdm(
    sample[start : len(sample)].itertuples(index=False),
    total=len(sample),
    initial=start,
    desc=f"Evaluating {model_name} on {dataset_name}: ",
):
    response = agent._evaluate_query(
        database=row.db_id, original_question=row.question, generated_sql_query=row.SQL
    )
    if response:
        if dataset_name == "bird":
            print(f"Difficulty: {row.difficulty} | Model Correct: {response["is_correct"]}")
        else:
            print(f"Model Correct: {response['is_correct']}")
        is_correct = response["is_correct"]
        successful_run = response["query_result"] != "error"
        feedback = response["feedback"]
    else:
        is_correct = False
        successful_run = False
        feedback = "LLM Failure"

    if dataset_name == "bird":
        evaluation.append({
            "question_id": row.question_id,
            "is_correct": is_correct,
            "difficulty": row.difficulty,
            "successful_run": successful_run,
            "feedback": feedback,
        })
    else:
        evaluation.append(
            {
                "question_id": row.question_id,
                "is_correct": is_correct,
                "successful_run": successful_run,
                "feedback": feedback,
            }
        )

    df = pd.DataFrame(evaluation)
    df.to_csv(
        f"./runs/feedback_agent/{dataset_name}/{model_name}/{model_type}.csv",
        index=False,
    )

Evaluating mistral on spider:  90%|######### | 90/100 [00:00<?, ?it/s]

Model Correct: True
Model Correct: True
Model Correct: False
Model Correct: True
Model Correct: True
Model Correct: True
Model Correct: True
Model Correct: True
Model Correct: True
Model Correct: True


In [8]:
accuracy = df["is_correct"].mean()
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 72.00%


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

if dataset_name == "bird":
    # Ensure all indices align with sorted difficulties
    difficulty_order = sorted(df["difficulty"].unique())

    # Calculate metrics, ensuring alignment
    accuracy_per_difficulty = (
        df.groupby("difficulty")["is_correct"]
        .mean()
        .reindex(difficulty_order, fill_value=0)
    )
    successful_examples = (
        df[df["successful_run"] == True]["difficulty"]
        .value_counts()
        .reindex(difficulty_order, fill_value=0)
    )
    unsuccessful_examples = (
        df[df["successful_run"] == False]["difficulty"]
        .value_counts()
        .reindex(difficulty_order, fill_value=0)
    )

    # Calculate total examples for consistency (or use successful + unsuccessful)
    examples_per_difficulty = successful_examples + unsuccessful_examples

    fig, ax1 = plt.subplots(figsize=(10, 6))  # Adjust figure size if needed

    # Plot accuracy per difficulty
    color = "tab:blue"
    ax1.set_xlabel("Difficulty")
    ax1.set_ylabel("Accuracy", color=color)
    ax1.bar(
        difficulty_order,
        accuracy_per_difficulty,
        color=color,
        label="Accuracy",
        width=0.3,
        align="center",
    )
    ax1.tick_params(axis="y", labelcolor=color)

    # Create a secondary y-axis to plot examples
    ax2 = ax1.twinx()
    ax2.set_ylabel("Number of Examples")

    # Plot examples and unsuccessful runs per difficulty
    width = 0.3  # Adjust bar width
    positions = range(len(difficulty_order))  # Shared x-axis positions

    # Stacked bar: successful and unsuccessful runs
    ax2.bar(
        [p + width for p in positions],
        successful_examples,
        color="tab:green",
        label="Successful Examples",
        width=width,
        align="center",
    )
    ax2.bar(
        [p + width for p in positions],
        unsuccessful_examples,
        bottom=successful_examples,
        color="tab:red",
        label="Unsuccessful Examples",
        width=width,
        align="center",
    )

    # Customize tick positions and labels
    ax1.set_xticks([p + width for p in positions])
    ax1.set_xticklabels(difficulty_order)

    ax1.grid(False)
    ax2.grid(False)

    # Adjust legend placement
    fig.legend(
        loc="upper left",
        bbox_to_anchor=(0.2, 0.9),
        bbox_transform=ax1.transAxes,
        frameon=False,
    )

    # Add title
    plt.title("Accuracy, Number of Examples, and Unsuccessful Runs per Difficulty")

    # Save the figure
    fig.savefig(f"runs/feedback_agent/{model_name}/{model_type}.png")

    fig.tight_layout()
    plt.show()

In [12]:
import os
import pandas as pd

# Define the base directory
base_dir = f"./runs/feedback_agent/{dataset_name}"

# Initialize an empty dictionary to store the data
data = {}

# Iterate through each model directory
for model in os.listdir(base_dir):
    model_dir = os.path.join(base_dir, model)
    if os.path.isdir(model_dir):
        data[model] = {}
        # Iterate through each CSV file in the model directory
        for csv_file in os.listdir(model_dir):
            if csv_file.endswith(".csv"):
                csv_path = os.path.join(model_dir, csv_file)
                prompt = os.path.splitext(csv_file)[0]
                # Read the CSV file into a DataFrame
                df = pd.read_csv(csv_path)
                data[model][prompt] = df

# Concatenate the DataFrames along a new axis
concat_data = {(model, prompt): df for model, prompts in data.items() for prompt, df in prompts.items()}

# Calculate accuracy and support for each model/prompt combination
accuracies = {key: df["is_correct"].mean() for key, df in concat_data.items()}
supports = {key: df["successful_run"].sum() for key, df in concat_data.items()}

# Convert the accuracies and supports dictionaries to DataFrames for better visualization
accuracy_df = pd.DataFrame(
    list(accuracies.items()), columns=["Model_Prompt", "Accuracy"]
)
support_df = pd.DataFrame(list(supports.items()), columns=["Model_Prompt", "Support"])

# Merge the accuracy and support DataFrames
merged_df = pd.merge(accuracy_df, support_df, on="Model_Prompt")

# Split the Model_Prompt column into separate Model and Prompt columns
merged_df[["Model", "Prompt"]] = pd.DataFrame(
    merged_df["Model_Prompt"].tolist(), index=merged_df.index
)
merged_df.drop(columns=["Model_Prompt"], inplace=True)

# Pivot the DataFrame to have models as rows, prompts as columns, and accuracy as values
pivot_df = merged_df.pivot(
    index="Model", columns="Prompt", values=["Accuracy", "Support"]
)

# Calculate the performance gain from worst to best
worst_accuracy = pivot_df["Accuracy"].min().min()
pivot_df["Performance Gain"] = (pivot_df["Accuracy"].max(axis=1) - worst_accuracy) * 100

pivot_df

,Accuracy,Support,Performance Gain
Prompt,two_shot,two_shot,
Model,,,
mistral,0.72,100.0,0.0
